학습-평가 데이터를 골고루 설정하여 모델의 안정성을 높이고 과대적합을 감소시키는 통계적 기법

#### k-fold

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold

x_data = np.array([
    [2,1],
    [3,2],
    [3,4],
    [5,5],
    [7,5],
    [2,5],
    [8,9],
    [9,10],
    [6,12],
    [9,2],
    [6,10],
    [2,4]
                  
])

In [2]:
kf = KFold(n_splits=5)
kf

KFold(n_splits=5, random_state=None, shuffle=False)

In [3]:
for train_index, test_index in kf.split(x_data):
    print(train_index)
    print(test_index)

[ 3  4  5  6  7  8  9 10 11]
[0 1 2]
[ 0  1  2  6  7  8  9 10 11]
[3 4 5]
[ 0  1  2  3  4  5  8  9 10 11]
[6 7]
[ 0  1  2  3  4  5  6  7 10 11]
[8 9]
[0 1 2 3 4 5 6 7 8 9]
[10 11]


#### K-Fold 교차검증 -> 보통 회귀 문제에서 사용
 - 학습 데이터와 테스트 데이터를 k개의 세트로 나누어 검증하는 방법
 - 데이터셋이 굉장히 적을 때 훈련데이터를 늘리기위해 사용되기도 하는 방법
 - 여러 개의 훈련 테스트 짝으로 검증과정을 거침

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [5]:
x_data = np.array([
    [2,1],
    [3,2],
    [3,4],
    [5,5],
    [7,5],
    [2,5],
    [8,9],
    [9,10],
    [6,12],
    [9,2],
    [6,10],
    [2,4]
                  
])

y_data = np.array([3, 5, 7, 10, 12, 7, 13, 13, 12, 13, 12, 6])

In [6]:
lr = LinearRegression()

In [7]:
train_scores = []
test_scores = []

kf = KFold(n_splits=5)

for train_index, test_index in kf.split(x_data):
    x_train = np.array(x_data)[train_index]
    y_train = np.array(y_data)[train_index]
    x_test = np.array(x_data)[test_index]
    y_test = np.array(y_data)[test_index]
    
    model_kf = LinearRegression()
    model_kf.fit(x_train, y_train)
    
    score = model_kf.score(x_train,y_train)
    train_scores.append(score)
    
    score = model_kf.score(x_test,y_test)
    test_scores.append(score)

In [8]:
train_scores,test_scores

([0.9522707858769932,
  0.9469593697441799,
  0.9446524178499608,
  0.9232432525564045,
  0.9166499001004778],
 [-1.1475590101753324,
  0.56847222331606,
  0.0,
  -11.7747639790487,
  0.9602035173350366])

In [9]:
np.array(train_scores).mean()

0.9367551452256032

In [10]:
np.array(test_scores).mean()

-2.278729449714587

In [11]:
from sklearn.model_selection import cross_validate

In [12]:
model = LinearRegression()

cv_result = cross_validate(model, x_data, y_data)

In [13]:
cv_result["test_score"].mean()

-2.278729449714587

In [14]:
df = pd.DataFrame(cv_result)
df = df.sort_values(by="test_score", ascending=False)
df

,fit_time,score_time,test_score
4,0.000,0.000000,0.960204
1,0.000,0.001000,0.568472
2,0.000,0.000000,0.000000
0,0.001,0.000000,-1.147559
3,0.000,0.001012,-11.774764


In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import r2_score

In [16]:
model = LinearRegression()


In [17]:
cv_score = cross_val_score(model, x_data, y_data, scoring="r2", cv=5)
cv_pred = cross_val_predict(model, x_data, y_data, cv=5)

In [18]:
cv_score.mean()

-2.278729449714587

In [19]:
r2_score(y_data,cv_pred)

0.7320211532492253

분류

In [20]:
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.linear_model import LogisticRegression

In [21]:
x_data = np.array([
    [2,1],
    [3,2],
    [3,4],
    [5,5],
    [7,5],
    [2,5],
    [8,9],
    [9,10],
    [6,12],
    [9,2],
    [6,10],
    [2,4]
                  
])

y_data = np.array([2, 2, 2, 1, 1, 2, 0, 0, 0, 1, 0, 2])

labels = ["A", "B", "C"]

In [22]:
model = LogisticRegression()

In [23]:
cv_result = cross_validate(model, x_data, y_data, return_estimator=True)

c:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(


In [24]:
cv_result["test_score"].mean(),cv_result["test_score"]

(0.9333333333333332,
 array([0.66666667, 1.        , 1.        , 1.        , 1.        ]))

#### 계층적 k-겹 교차검증 (Stratified k-fold cross validation)

 - 분류 모델에 적용
 - k-겹 교차검증 모델은 k-fold가 원본 데이터 집합의 레이블 분포를 학습 및 검증데이터 세트에 제대로 분배하지 못하는 문제를 해결해줌
    - 데이터의 분포가 불균형 할때 제대로된 검증을 하지 못할수 있음
 - target 을 레이블/클래스 값을 기준으로 골고루 넣어주게 됨

In [25]:
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris

In [38]:
iris = load_iris()
model = LogisticRegression(max_iter= 500)

In [39]:
x = iris.data
y = iris.target

In [40]:
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

In [41]:
idx_iter = 0
cv_accuracy = []

for train_index, test_index in skf.split(x, y):
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    
    idx_iter += 1
    
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print(f"{idx_iter} 번째 교차검증 정확도 : {accuracy} \n",
          f"학습 데이터 크기 : {train_size} \n",
          f"검증 데이터 크기 : {test_size}")
    
    
    cv_accuracy.append(accuracy)

1 번째 교차검증 정확도 : 1.0 
 학습 데이터 크기 : 120 
 검증 데이터 크기 : 30
2 번째 교차검증 정확도 : 0.9667 
 학습 데이터 크기 : 120 
 검증 데이터 크기 : 30
3 번째 교차검증 정확도 : 0.9333 
 학습 데이터 크기 : 120 
 검증 데이터 크기 : 30
4 번째 교차검증 정확도 : 1.0 
 학습 데이터 크기 : 120 
 검증 데이터 크기 : 30
5 번째 교차검증 정확도 : 0.9333 
 학습 데이터 크기 : 120 
 검증 데이터 크기 : 30
